In [1]:
import obspy, os
import pandas as pd
import numpy as np
from obspy.geodetics.base import gps2dist_azimuth
from scipy.interpolate import interp2d


In [2]:
WD_new_lat_lon = pd.read_csv('WD_new.csv',names=['lon','lat'])
WD_bty = pd.read_csv('wd_ChaoJing_new.csv',header=None)
GPStime = pd.read_csv('gpstime.csv',names=['time'])
GPSlat = pd.read_csv('gps_lat.csv',names=['AUV','Buoy','Ship'])
GPSlon = pd.read_csv('gps_lon.csv',names=['AUV','Buoy','Ship'])

In [3]:
envfile = 'Buoy-Ship'

freq = 18500.000
Nmedia = 1 
topopt = 'CVW'
option2='A*'
cpbottom = 2300
csbottom = 0.00
den_bottom = 2.300
alpha_bottom = 1.200
Nbeams = 1001
alpha = 20.0
step = 1.000
zbox = 50.000

In [11]:
def createBTY(lentobty, Shipbty, sorted_Dis,sorted_Bty,runtype,rbox, savepath, envfile,texttime):
    if runtype == 'E':
        end = 'eigen'
    elif runtype =='A':
        end = 'amp'
    ##Create bty file 
    cmd = ''' echo $'\"C\"' > %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd) 
    cmd = '''echo %(lentobty)s  >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd) 
    Shipbty = abs(Shipbty)
    if Shipbty> max(abs(sorted_Bty)):
        Shipbty = max(abs(sorted_Bty))
    cmd = ''' echo 0.000000 %(Shipbty)f >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd)
    for i in range(lentobty-1):
        dist = sorted_Dis[i]
        bty  = -1*sorted_Bty[i]
        cmd  = ''' echo %(dist)f %(bty)f >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty ''' %locals()
        os.system(cmd)
    cmd = ''' echo  >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd)

In [5]:
def createENV(savepath, envfile,texttime, titleenv,freq,Nmedia, topopt, BtyLine,option2,cpbottom, csbottom, den_bottom,alpha_bottom, \
        NSD,SD,NRD,RD,NRR,RR,runtype,Nbeams,alpha,step,zbox,rbox):
    if runtype == 'E':
        end = 'eigen'
    elif runtype =='A':
        end = 'amp'
    deepest = int(abs(min(BtyLine)))+1
    file = f'{savepath}/{envfile}_time{texttime}_{end}.env'
    f = open(file, 'w')
    f.write(f"' {titleenv} ' \n")
    f.write(f'{freq} \n')
    f.write(f'{Nmedia} \n')
    f.write(f"'{topopt}' \n")
    f.write(f'50 0.00 {deepest} \n')
    f.write(f'    0.000000  1533  /\n')
    f.write(f'    {deepest}  1533  /\n')
    f.write(f"'{option2}'  0.0 \n")
    f.write(f' {deepest} {cpbottom} {csbottom} {den_bottom} {alpha_bottom} / ! lower halfspace \n')
    f.write(f'{NSD}			! NSD \n')
    f.write(f'{SD}  ! SD(1:NSD)  ... \n')
    f.write(f'{NRD} /		! NRD \n')
    f.write(f'{RD}   /		! RD(1:NRD)  ... \n')
    f.write(f'{NRR} /		! NRR \n')
    f.write(f'{RR}   /		! RR(1)  ... \n')
    f.write(f"'{runtype}' \n")
    f.write(f'{Nbeams} \n')
    f.write(f'{-alpha} {alpha} / \n')
    f.write(f'{step} {zbox} {rbox} \n')
    f.close()



## interp 2d to find water depth

In [6]:
f = interp2d(WD_new_lat_lon['lon'][0:169],WD_new_lat_lon['lat'],WD_bty.T,kind='linear',fill_value=True)

In [12]:
import matplotlib.pyplot as plt

for itime in range(1,2): #range(len(GPStime)):
    texttime = str(itime).zfill(3)
    savepath = f'time_{texttime}'
    if not os.path.isdir(savepath):
        os.makedirs(savepath)
    Ship_lat = 25+GPSlat['Ship'].values[itime]/60
    Ship_lon = 121+GPSlon['Ship'].values[itime]/60
    Buoy_lat = 25+GPSlat['Buoy'].values[itime]/60
    Buoy_lon = 121+GPSlon['Buoy'].values[itime]/60
    # plt.plot([Ship_lon,Buoy_lon],[Ship_lat,Buoy_lat],'ro')
    Shipbty = f(Ship_lon,Ship_lat)[0]
    print(f'Ship depth: {Shipbty}m')
    Buoybty = f(Buoy_lon,Buoy_lat)[0]
    print(f'Buoy depth: {Buoybty}m')

    dist,az,baz = gps2dist_azimuth(Ship_lat,Ship_lon,Buoy_lat,Buoy_lon)
    ttt = 20
    dx = dist/ttt
    BtyLine = [] ; SourceDist=[]
    for i in range(1,ttt):
        distance = dx * i 
        if distance <= dist : 
             newlat = Ship_lat + (distance* np.cos(az*np.pi/180))/111/1000
             newlon = Ship_lon + (distance* np.sin(az*np.pi/180))/111/1000
             interpoint_depth = f(newlon,newlat)[0]
             BtyLine.append(round(interpoint_depth,3))
             SourceDist.append(round(distance/1000,5))
            #  print(interpoint_depth)

    Dist_array = np.array(SourceDist)
    Bty_array = np.array(BtyLine)
    inds = Dist_array.argsort()
    sorted_Bty = Bty_array[inds]
    sorted_Dis = sorted(Dist_array)

    print(f'Max distance: {max(sorted_Dis)}km')

    
    lentobty = int(len(sorted_Dis))+1
    titleenv = f'ChaoJing {envfile} {texttime}'
    NSD = 1 ##number of sourece depth
    SD = 10 ##source depth
    NRD = 1 ##number of receiver depth
    RD = 10 ##receiver depth
    NRR  = 1 ## the number of receiver ranges
    RR = max(sorted_Dis) ##receiver range
    rbox = max(sorted_Dis)+0.5

    createBTY(lentobty, Shipbty, sorted_Dis,sorted_Bty,'E',rbox, savepath, envfile, texttime)
    createBTY(lentobty, Shipbty, sorted_Dis,sorted_Bty,'A',rbox, savepath, envfile, texttime)

    createENV(savepath, envfile,texttime, titleenv,freq,Nmedia, topopt, BtyLine,option2,cpbottom, csbottom, den_bottom,alpha_bottom, \
        NSD,SD,NRD,RD,NRR,RR,'E',Nbeams,alpha,step,zbox,rbox)
    createENV(savepath, envfile,texttime, titleenv,freq,Nmedia, topopt, BtyLine,option2,cpbottom, csbottom, den_bottom,alpha_bottom, \
        NSD,SD,NRD,RD,NRR,RR,'A',Nbeams,alpha,step,zbox,rbox)

Ship depth: -33.88249315606257m
Buoy depth: -30.595745145990538m
Max distance: 0.36936km
